# Getting started
In this notebook, we demonstrate how to load the *i*CH360 metabolic model and perform basic stoichiometric analysis, such as Flux Balance Analysis (FBA), Flux Variability Analysis (FVA) and parsimonious Flux Balance Analayis (pFBA). We also demonstrate how to export a flux distribution for visualisation the the *i*CH360 model maps in Escher (https://escher.github.io/#/).

All analysis will be performed using the popular metabolic modelling toolbox COBRApy. You can find many more turorials/examples of model manipulation in the COBRApy documentation (https://cobrapy.readthedocs.io/). 

Let's start by importing the required packages

In [1]:
import cobra #for model manipulation/analysis
import json  #to export fluxes for visualisation in Escher
import sys #To import the custom script for compressed map visualisation

## Loading the model

First let's read *i*CH360 from file using the COBRA toolbox

In [2]:
model=cobra.io.read_sbml_model('../Model/iCH360/Escherichia_coli_iCH360.xml')
model

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12


Name,iCH360
Memory address,1c17acc2da0
Number of metabolites,304
Number of reactions,349
Number of genes,356
Number of groups,0
Objective expression,1.0*Biomass - 1.0*Biomass_reverse_57a34
Compartments,"cytosol, extracellular space, periplasm"


## Flux Balance Analysis

By default, the model is set up to model aerobic growth on glucose as a sole carbon source, with an uptake bound of 10 mmol/gDW/h for glucose and no restrictions on oxygen uptake. The default objective of the model is the Biomass reaction (growth rate).


In [3]:
print(f"Glucose exchange reaction (EX_glc__D_e) bounds: {model.reactions.get_by_id('EX_glc__D_e').bounds}")
print(f"Oxygen exchange reaction (EX_o2_e) bounds: {model.reactions.get_by_id('EX_o2_e').bounds}")
print('----')
print(f"Model objective: {model.objective}")

Glucose exchange reaction (EX_glc__D_e) bounds: (-10.0, 1000.0)
Oxygen exchange reaction (EX_o2_e) bounds: (-1000.0, 1000.0)
----
Model objective: Maximize
1.0*Biomass - 1.0*Biomass_reverse_57a34


Let's run Flux Balance Analysis for this growth condition, using the `cobra.Model.optimize()` method.

In [4]:
fba_sol_glc_aer=model.optimize()

print(f"Status: {fba_sol_glc_aer.status}. Optimal objective: {round(fba_sol_glc_aer.objective_value,3)}/h")
fba_sol_glc_aer.fluxes.head(2)

Status: optimal. Optimal objective: 0.866/h


NDPK5     0.023387
SHK3Dr    0.328942
Name: fluxes, dtype: float64

If we want to run FBA for a different growth condition, we simply block glucose uptake and lift the lower bound on the uptake of the desired carbon source and/or Oxygen,. For example, let's run FBA for anaerobic growth on fructose

In [5]:
with model as m:
    #Block uptake of glucose (default carbon source)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound=0
    #Enable uptake of pyruvate (up to 10 mmol/gDW/h)
    model.reactions.get_by_id('EX_fru_e').lower_bound=-10
    #Block uptake of oxygen to simulate anaerobic conditions
    model.reactions.get_by_id('EX_o2_e').lower_bound=0

    #run FBA
    fba_sol_fru_an=m.optimize()
    
print(f"Status: {fba_sol_fru_an.status}. Optimal objective: {round(fba_sol_fru_an.objective_value,3)}/h")
fba_sol_fru_an.fluxes.head(2)

Status: optimal. Optimal objective: 0.157/h


NDPK5     0.004230
SHK3Dr    0.059501
Name: fluxes, dtype: float64

Note that the use of the "with" construct, also known as context, enables us to make temporary changes (like, in this case, changing reaction bounds) without modifying the original model object.

## Flux Variability Analysis and Parsimonious Flux Balance Analysis

A common limitation of Flux Balance Analysis is that the optimal solution may not be unique. That is, there may exist multiple flux distributions achieving the same optimal objective (i.e. in this case, achieving the same optimal growth rate). In practice, the solution we obtained above is just *one* of these equally optimal flux distributions, and which solution is returned depends largely on the specific implementation of the solver. To assess the extent of this degeneracy at the optimum, a popular technique is Flux Variability Analysis (FVA). In simple terms, one can think of FVA as a way of "pooling" together all equally optimal solutions and assessing how much the flux for each reaction varies across them.

Let's run FVA. Setting the `loopless=True` ensures that the solver excludes thermodynamically infeasible loops from the solution, resulting in more realistic results.

In [6]:
#Run FVA
fva_sol=cobra.flux_analysis.flux_variability_analysis(model,loopless=True)

#Compute variability (gap between max and min flux at the optimum)
fva_sol['variability']=fva_sol['maximum']-fva_sol['minimum']

#Inspect the first 5 occurrences
fva_sol.sort_values('variability',ascending=False).head(5)

,minimum,maximum,variability
THD2pp,0.000000,4.891289,4.891289
GND,0.511807,3.446580,2.934773
G6PDH2r,0.511807,3.446580,2.934773
PGL,0.511807,3.446580,2.934773
PGI,6.553420,9.488193,2.934773


As you can see, the flux through some reaction can vary at the optimum, meaning that the optimal FBA solution is indeed not unique (though the contained size of *i*CH360 does not allow for a whole lot of variability). A popular heuristic to resolve this degeneracy in stoichiometric modelling is to introduce additional optimisation criteria on top of the FBA objective. For example, parsimonious FBA (pFBA), resolves ambiguity at the optimum by selecting, among equally optimal flux distributions, the one with the smallest sum of absolute fluxes.

Let's run pFBA and verify that the, while the optimal flux distribution may be different from the one computed through FBA, the optimal objective (in this case, the flux through the Biomass reaction), is indeed the same as the FBA solution.

In [7]:
pfba_sol=cobra.flux_analysis.pfba(model)
print(f"Optimal growth rate:\nFBA:  {round(fba_sol_glc_aer.fluxes['Biomass'],3)},\npFBA: {round(pfba_sol.fluxes['Biomass'],3)}"
)

Optimal growth rate:
FBA:  0.866,
pFBA: 0.866


## Visualising flux distributions in Escher

A great advantage of medium-scale models, such as *i*CH360, is that flux distributions through the network can be visualised in their entirety via metabolic visualisation tools such as Escher. Escher works with `JSON` data, so we will first export our flux distribution in this format.Let's work, for example, with the pFBA solution we just computed.

In [8]:
flux_distribution=pfba_sol.fluxes
flux_distribution.head()

NDPK5     0.023387
SHK3Dr    0.328942
NDPK6     0.022650
NDPK8     0.022650
DHORTS   -0.286345
Name: fluxes, dtype: float64

We will write this distribution to file with the `json` package. Note that COBRApy solutions are typically `pandas.Series` objects, but they will need to be converted to a python dictionary prior to saving to file.

In [9]:
#Open the desired output file in write ('w') mode
with open('out/escher_visualisation/glc_aer_pfba_fluxes.json','w') as file:
    #Write the flux distribution to file in JSON format
    #Note the use of the to_dict() method to convert the Series into a python dictionary
    json.dump(flux_distribution.to_dict(),file)

Let's now visualise the flux distribution in Escher. First, go to the Escher website (https://escher.github.io/#/) and open an empty visualisation canvas by selecting `None` in the "map" dropdown and `viewer` in the "Tool" dropdown, followed by presing the "Load map" button.

<img src="data/escher_visualisation/escher_front_page.png" width="800">

Then, use Map->Load Map JSON and select the *i*CH360 metabolic map (`../Visualisation/model_map/full_map.json`)

<img src="data/escher_visualisation/escher_load_map.png" width="600">

Once the map has loaded, use Data -> Load reaction data and select the `.json` file we just exported to load the flux distribution.

<img src="data/escher_visualisation/escher_load_reaction_data.png" width="1000">


You should now be able to see fluxes overlaid onto the map!

<img src="data/escher_visualisation/escher_flux_distribution_viz.png" width="1000">

### Using the compressed map
In addition to the full map we just used, *i*CH360 also comes with a condensed map, wherein linear pathways have been lumped into individual reactions. Because these lumped reactions are only visualisation convenience and don't belong to the model, you cannot directly use the model solution onto this map.

If you wish to use it, fluxes must first be mapped to the ones in the map. Luckily, we took care of this mapping, and provde a function that automatically maps and export fluxes for the compressed map. You can find this function in `../Visualisation/visualisation_utils.py`. Here is an example demonstrating this possibility.

In [10]:
#Import the custom function
sys.path.append('../Visualisation')
from visualisation_utils import export_fluxes_for_compressed_map

#Map and export the fluxes in the chosen file
export_fluxes_for_compressed_map(fluxes=flux_distribution,
                                 output_file='out/escher_visualisation/glc_aer_pfba_fluxes_compressed.json')

You can now load the compressed map (`../Visualisation/model_map/compressed_map`) in Escher as we did above, and load the `.json` file we just generated as reaction data:

<img src="data/escher_visualisation/escher_flux_distribution_compressed_map_viz.png" width="1000">